## Line sweep
#### TODO 986 还没有做
#### TODO 57 range module 还有meta 的面试题 再斟酌一下

#### 391. Number of Airplanes in the Sky (LintCode)
#### 252. Meeting Rooms
给定一个时间区间的列表，求是否能参加所有的meeting
i.e.给定一个区间的列表，求是否有overlap的区间
252 Given an array of meeting time intervals consisting of start and end times $[[s 1, e 1],[s 2, e 2], \ldots]\left(s_i<e_i\right)$, determine if a person could attend all meetings.
252 $O(nlogn)$
#### 253. Meeting Rooms II
给定一个区间的列表，求最大的同时overlap的区间数
Given an array of meeting time intervals intervals where intervals[i] = [starti, endi], return the minimum number of conference rooms required.
253 O(nlogn)
```
# 1 sort the interval by start
    intervals.sort(key=lambda x: x[0])
# 1.5 initialize heap
hq = [float("inf")]
heapq.heapify(hq)
# 2 start from the first interval
cnt = 0
for idx, (s,t) in enumerate(intervals)
# when we meet the next interval, check if any previous meeting room occupied can be evacuated. (-> start_nex > end_pre(s)), evacuate those meeting rooms
# Using heap to store ending time, pop the least ones until it > start_nex
while hq and hq[0] <= start_nex:
    heapq.heappop(hq)

# add the cur_end
heapq.heappush(hq, end_nex)
# update max count of meeting room
cnt = max(len(hq)-1, cnt)
# return cnt
```
#### 56. Merge Intervals
Given a set of non-overlapping intervals, insert a new interval into the intervals (merge if necessary).
You may assume that the intervals were initially sorted according to their start times.
```
class Solution:
    def merge0(self, intervals: List[List[int]]) -> List[List[int]]:

        intervals.sort(key=lambda x:x[0])
        merged = []
        sm, tm = intervals[0]
        for s, t in intervals:
            if s > tm: #[1,3] merged [4,6]
                merged.append([sm, tm])
                # update sm
                sm = s
                tm = t
            elif s <= tm and t > tm: #[2,4] merged [3,7] 追忆 [2,4] merged [3,3]的情况 要判断 t > tm
                tm = t # tm = max(t, tm)

        merged.append([sm, tm]) # the last [sm, tm]
        return merged

    def merge(self, intervals: List[List[int]]) -> List[List[int]]:

        intervals.sort(key=lambda x:x[0])
        merged = []
        # for each interval we iterate
        for idx, (s, t) in enumerate(intervals):
        # put [s, t] in merged and see if we need to modify the interval or just append it at the tail
            # [s, t] overlap with the last interval in merged
            if len(merged) >= 1:
                s1, t1 = merged[-1] 
                if s <= t1:
                    merged[-1] = [min(s, s1), max(t, t1)]
                else:
                    merged.append([s, t])
            else:
                merged.append([s, t])
        
        return merged

```
#### 57. Insert Interval
给定一个不overlap的区间列表，插入新的区间使得新的区间列表俩俩也不overlap
Given a set of non-overlapping intervais, insert a new interval into the intervals (imerge if necessary).
给你一个 无重叠的 ，按照区间起始端点排序的区间列表 intervals，其中 intervals[i] = [starti, endi] 表示第 i 个区间的开始和结束，并且 intervals 按照 starti 升序排列。同样给定一个区间 newInterval = [start, end] 表示另一个区间的开始和结束。

在 intervals 中插入区间 newInterval，使得 intervals 依然按照 starti 升序排列，且区间之间不重叠（如果有必要的话，可以合并区间）。

返回插入之后的 intervals。

注意 你不需要原地修改 intervals。你可以创建一个新数组然后返回它。
``` draft (correct but messy)   
    def insert(self, intervals, newInterval):

        
        s, t  = newInterval
        I = intervals
        res = I
        i = bisect.bisect_left(I, [s])
        i1 = i
        # edge case: if i == len(I)-1
        if i == 0: 
            start = s
            i1 = i
        elif i > 0:
            start = s if s > I[i-1][1] else I[i-1][0]
            i1 = i if start == s else i-1

        j = bisect.bisect_right(I, [t, float("inf")])
        if j == 0:
            end = t
        elif j > 0:
            end = t if t > I[j-1][1] else I[j-1][1]
        

        if i1 < j:
            res[i1: j] = [[start, end]]
        elif i1 == j and i1 != 0 and i1!=len(I):
            res = res[:i1] + [[start, end]] + res[j:]
        elif i1 == 0:
            res = [[start, end]] + res
        elif i1 == len(I):
            res.append([start, end])

        return res
```

```
# 类似range module的正确答案
def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
    start_intervals = [interval[0] for interval in intervals]
    end_intervals = [interval[1] for interval in intervals] # 因为是non overlap end points 也是sorted的
    start_range = bisect_left(end_intervals, newInterval[0])
    end_range = bisect_right(start_intervals, newInterval[1])
    if start_range ==len(intervals):
        return intervals[0:start_range] + [newInterval]
    if end_range ==0:
        return [newInterval] + intervals
    res = intervals[0:start_range] + [[min(newInterval[0],intervals[start_range][0]), max(newInterval[1],intervals[end_range-1][1])]] + intervals[end_range:]
    return res
```
```
# 标准答案
def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
    res = []

    for i, n in enumerate(intervals):
        if newInterval[1] < n[0]:
            res.append(newInterval)
            return res + intervals[i:]

        elif newInterval[0] > n[1]:
            res.append(n)

        else:
            newInterval = [min(newInterval[0], n[0]), max(newInterval[1], n[1])]

    res.append(newInterval)

    return res

```

``` 一开始写的错误答案
# find the insert point of [s] and [t, float("inf")]
res = I
i = bisect.bisect_left(I, [s])
# edge case: if i == len(I)-1
if i == 0: 
    start = s
elif i > 0:
    start = s if s > I[i-1][1] else I[i-1][0] # 其实start==s时并不会从i-1开始insert 而是从i开始 应该设i1


j = bisect.bisect_right(I, [t, float("inf")])
if j == 0:
    end = t
elif j > 0:
    end = t if t > I[j-1][1] else I[j-1][1]

res[i-1: j] = [[start:end]]

return res
```


You may assume that the intervals were initially sorted according to their start times.
#### 1272. Remove Interval
Given a sorted list of disjoint intervals, each interval intervals $[i]=[a, b)$ represents the set of real numbers $x$ such that $\mathrm{a}<=\mathrm{x}<\mathrm{b}$.

We remove the intersections between any interval in intervals and the interval toBeRemoved.
Return a sorted list of intervals after all such removals.

#### Range Module (Hard)
https://leetcode.com/problems/range-module/description/
code一个类去track所有半开半闭区间之中的所有整数。addRange表示加入区间 queryRange表示查询区间的整数是否在tracking中， removeRange表示去除区间

```
class RangeModule:

    def __init__(self):
        self.track = []

    def addRange(self, left, right):
        start = bisect.bisect_left(self.track, left)
        end = bisect.bisect_right(self.track, right)
        
        subtrack = []
        if start % 2 == 0:
            subtrack.append(left)
        if end % 2 == 0:
            subtrack.append(right)
			
        self.track[start:end] = subtrack

    def removeRange(self, left, right):
        start = bisect.bisect_left(self.track, left)
        end = bisect.bisect_right(self.track, right)
        
        subtrack = []
        if start % 2 == 1:
            subtrack.append(left)
        if end % 2 == 1:
            subtrack.append(right)
			
        self.track[start:end] = subtrack
		
    def queryRange(self, left, right):
        start = bisect.bisect_right(self.track, left)
        end = bisect.bisect_left(self.track, right)
		
        return start == end and start % 2 == 1


class RangeModule:

    def __init__(self):
        self.I =[]
        

    def addRange(self, left: int, right: int) -> None:
        start = bisect.bisect_left(self.I, left)
        end = bisect.bisect_right(self.I, right)
        subI = []

        # case 1 if left is in a certain range [i, j), start = index of i (new left = i)
        # case 2 if left is not in a certain range [j, i), start replace the next left end, i.e. index of i  (new left = left)
        # 
        if start %2 == 0: # %2 == 0 left_end of interval, i.e. case 2
            subI.append(left)
        # case 1 if right is in a certain range [i, j), end = index of j = index of i+1, new right = j
        # case 2 if right is not in a certain range [j, i), end replace the previous right end, [:index of j + 1], i.e. index of i, [:index of i]
        if end % 2 ==0: # %2 == 0 left_end of interval, i.e. case 2
            subI.append(right)
        
        self.I[start: end] = subI


    def queryRange(self, left: int, right: int) -> bool:
        start = bisect.bisect_right(self.I, left)
        end = bisect.bisect_left(self.I, right)

        return start == end and start%2 == 1
        

    def removeRange(self, left: int, right: int) -> None:
        start = bisect.bisect_left(self.I, left)
        end = bisect.bisect_right(self.I, right)
        subI = []

        # reverse of addRange
        # if start is in a certain range [i, j), start replace j
        # if start is not in a certain range [j, i), start is replaced by the previous range's right end, i.e. j
        if start %2 == 1:
            subI.append(left)

        if end %2 == 1:
            subI.append(right)

        self.I[start: end] = subI


# Your RangeModule object will be instantiated and called as such:
# obj = RangeModule()
# obj.addRange(left,right)
# param_2 = obj.queryRange(left,right)
# obj.removeRange(left,right)

```

#### 435. Non-overlapping Intervals (Remove Overlapped Intervals)
Given an array of intervals intervals where `intervals[i] = [starti, endi]`, return the minimum number of intervals you need to remove to make the rest of the intervals non-overlapping. 移除最少的interval使得剩下的interval不会overlap，问要
移除多少interval

Greedy 当第一个interval包含当前interval时 我们移除更有可能和后面的interval overlap的一个（反正必须移除一个）

Example 1:

Input: intervals = [[1,2],[2,3],[3,4],[1,3]]
Output: 1
Explanation: [1,3] can be removed and the rest of the intervals are non-overlapping.

```
class Solution:
    def eraseOverlapIntervals(self, intervals: List[List[int]]) -> int:
        
        
        if len(intervals)<=1:
            return 0
        
        intervals.sort(key = lambda x: x[0])
        cnt = 0
        
        s0, t0 = intervals[0]
        
        for i in range(1,len(intervals)):
            s1, t1 = intervals[i]
            
            if s1<t0:
                cnt += 1
                if t0 > t1: # 当第一个interval包含当前interval时 我们移除更大的一个 更有可能和后面的interval overlap的一个（反正必须移除一个）
                    # remove [s0, t0]
                    s0, t0 = s1, t1
                # else 我们移除 [s1, t1]
            else:
                s0, t0 = s1, t1
        
        return cnt

```


#### 1288. Remove Covered Intervals
Given一个区间列表，去除所有被其它区间cover的子区间 使得剩下的区间都不会互相cover（但可以overlap）
key: interval cover 的equivalent conditions
把interval按start排序，只可能是前面的interval包含后面的，因为后面的start一定小于前面的
一前一后的两个interval `(i<j)` 要判断是否i是否coverj, 只需判断`i.end ?>= j.end`
反过来 j也可能cover i，当`i.start==j.start`时， `i.end<j.end`意味着`i covered by j`
也就是说 `i.end< j.end` 并不能说明  `i和j不互相cover`

但是此时若把interval先按start升序后按end降序排列，则`i.end < j.end` 可以等价于 `i j 不互相cover`
```
class Solution:
    def removeCoveredIntervals(self, intervals: List[List[int]]) -> int:
        intervals.sort(key = lambda x: x[0])

        cur_s, cur_t = intervals[0]
        res = [[cur_s, cur_t]]
        for i in range(1, len(intervals)):
            s, t = intervals[i]
            cur_s, cur_t= res[-1]
            in_s, in_t = max(cur_s, s), min(cur_t, t)
            if [in_s, in_t] == [cur_s, cur_t]: # cur_s, cur_t is coverd by s, t
                # remove [cur_s, cur_t]
                res.pop()
                res.append([s, t])
            elif [in_s, in_t] == [s, t]:
                # I won't add [s, t] to res
                continue
            else:
                res.append([s, t])

        return len(res)

```

#### 352. Data Stream as Disjoint Intervals (Hard)
Given 一个数据流data stream，stream中是整数，把数据流归并成不相交的区间 disjoint intervals
Key: equivalent conditions
lst = [0 for _ in range(l)]
interval needs to be merged iff sum(lst[i:j]) == j-i
for each number added in
get the interval that covered it
lb[num] = num -1 or lb[num-1]
rb[num] = num +1 or rb[num+1]
```
class SummaryRanges:

    def __init__(self):
        self.lb = [0 for _ in range(10**4+1)]
        self.rb = [0 for _ in range(10**4+1)]
        self.intervals = []

    def addNum(self, value: int) -> None:
        if value > 0 or value < 10**4:
            self.lb[value] = value if self.lb[value-1] == 0 else self.lb[value-1]
            self.rb[value] = value if self.rb[value+1] == 0 else self.rb[value+1]
        elif value == 0:
            self.lb[value] = 0
            self.rb[value] = value if self.rb[value+1] == 0 else self.rb[value+1]
        elif value == 10**4:
            self.lb[value] = value if self.lb[value-1] == 0 else self.lb[value-1]
            self.rb[value] = value
        
        # update lb, rb
        for v in range(self.lb[value], self.rb[value]+1):
            self.lb[v] = self.lb[value]
            self.rb[v] = self.rb[value]

        
        # update self.intervals
        # merge interval [lb[value], rb[value]] into self.intervals
        lidx = bisect.bisect_left(self.intervals, [self.lb[value]])
        ridx = bisect.bisect_right(self.intervals, [self.rb[value], self.rb[value]])
        self.intervals[lidx:ridx] = [[self.lb[value], self.rb[value]]]

    def getIntervals(self) -> List[List[int]]:
        return self.intervals
        


# Your SummaryRanges object will be instantiated and called as such:
# obj = SummaryRanges()
# obj.addNum(value)
# param_2 = obj.getIntervals()

```

#### 1229. Meeting Scheduler (返回最早的common time interval、有长度限制)
Given 两个person 的 availability time list（区间列表），找出最早的长为 duration 的 earlist slot 使得两人都是available。其实就是找出最早start的满足duration长度的intersection
+ 如何求overlapped interval
+ 如何取舍和移动
  + 移动end较少的那个person的pointer
```
class Solution:
    def minAvailableDuration(self, slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
        
        slots1.sort()
        slots2.sort()
        # step 1 set two points
        p1, p2 = 0, 0
        # get intersection from interval in slots1 and interval in slots2
        while p1 <= len(slots1)-1 and p2 <= len(slots2)-1:
            l1, r1, l2, r2 = slots1[p1][0], slots1[p1][1], slots2[p2][0], slots2[p2][1]
            il, ir = max(l1, l2), min(r1, r2)
            if ir - il >= duration:
                return [il, il+duration] #return [il, ir] 
            
            # move p1 and p2
            if r1 <= r2 :
                p1 += 1
            else:
                p2 += 1
        
        return []
```

#### 986. Interval List Intersections (返回所有的common slot. 无长度限制)
Given 两列区间列表，列表中的区间都是俩俩不相交且递增排列。求两个区间列表之间区间的相交列表。
```
class Solution:
    def intervalIntersection(self, A: List[List[int]], B: List[List[int]]) -> List[List[int]]:
        
        a = 0
        b = 0
        ans = []
        
        if len(A)*len(B) == 0:
            return ans
        
        while a < len(A) and b < len(B):
            
            [al, ar] = A[a]
            [bl, br] = B[b]
            cap = [max(al,bl), min(ar,br)]
        
            if cap[1] - cap[0] >=0:
                ans.append(cap)

            if ar< br:
                a += 1
            elif ar == br:
                a += 1
                b += 1
            else:
                b += 1
        
        return ans

```

#### 759. Employee Free Time
Given 区间列表的列表schedule, 每个intervals列表对应一个员工的休息时间安排. 求所有员工的共有休息时间 返回该区间列表。
```
# Employee Free Time
from collections import defaultdict
def solution(schedule):
    # TODO 尝试一下 interval intersection 的办法
    # method 1 sweep line
    d_schedule = defaultdict(int)
    for emp, ins in enumerate(schedule):
        if ins == []: return []
        for s, t in ins:
            d_schedule[s] += 1
            d_schedule[t] -= 1

    precnt, cnt = 0, 0
    res = []
    cs, ct = -1, 0
    for time, occupy in sorted(d_schedule.items()):
        cnt += occupy
        if precnt == 0 and cnt > 0:
            ct = time
            if ct > cs and cs != -1:
                res.append([cs, ct])
        elif cnt == 0:
            cs = time
        precnt = cnt
    return res

intervals = [[[1,2],[5,6]],[[1,3]],[[4,10]]]
intervals = [[[1,3],[6,7]],[[2,4]],[[2,5],[9,12]]]
intervals = [[[45,56],[89,96]],[[5,21],[57,74]]]
res = solution(intervals)
print(res)

```

#### 218. The Skyline Problem
给定一个building列表，每个term是[左端点 右端点 高度] 三元组， 求所有轮廓上高度变化的右端点
```
class Solution:
    def getSkyline(self, buildings):
        """
        :type buildings: List[List[int]]
        :rtype: List[List[int]]
        """
        # 对于一个 building, 他由 (l, r, h) 三元组组成, 我们可以将其分解为两种事件: 
        #     1. 在 left position, 高度从 0 增加到 h(并且这个高度将持续到 right position); 
        #     2. 在 right position, 高度从 h 降低到 0. 
        # 由此引出了 event 的结构: 在某一个 position p, 它引入了一个高度为 h 的 skyline, 将一直持续到另一 end postion
        
        # 对于在 right position 高度降为 0 的 event, 它的持续长度时无效的
        # 只保留一个 right position event, 就可以同时触发不同的两个 building 在同一 right position 从各自的 h 降为 0 的 event, 所以对 right position events 做集合操作会减少计算量
        
        # 由于需要从左到右触发 event, 所以按 postion 对 events 进行排序
        # 并且, 对于同一 positon, 我们需要先触发更高 h 的事件, 先触发更高 h 的事件后, 那么高的 h 相比于低的 h 会占据更高的 skyline, 低 h 的 `key point` 就一定不会产生; 相反, 可能会从低到高连续产生冗余的 `key point`
        # 所以, event 不仅需要按第一个元素 position 排序, 在 position 相同时, 第二个元素 h 也是必须有序的
        events = sorted([(l, -h, r) for l, r, h in buildings] + list({(r, 0, 0) for l, r, h in buildings}))
        
        # res 记录了 `key point` 的结果: [x, h]
        # 同时 res[-1] 处的 `key point` 代表了在下一个 event 触发之前, 一直保持的最高的 skyline
        
        # hp 记录了对于一条高为 h 的 skyline, 他将持续到什么 position 才结束: [h, endposition]
        # 在同时有多条 skyline 的时候, h 最高的那条 skyline 会掩盖 h 低的 skyline, 因此在 event 触发时, 需要得到当前最高的 skyline
        # 所以利用 heap 结构存储 hp, 它的第一个值永远为列表中的最小值: 因此在 event 中记录的是 -h, heap 结构就会返回最高的 skyline. 同时, h 必须在 endposition 之前, 因为它按第一个元素排序
        res, hp = [[0, 0]], [(0, float('inf'))]

        for l, neg_h, r in events:
            # 触发 event 时, 首先要做的就是清除已经到 endposition 的 skyline
            # hp: [h, endposition]
            # 如果当前 position 大于等于了 hp 中的 endposition, 那么该 skyline 会被清除掉
            # 由于在有 high skyline 的情况下, low skyline 不会有影响, 因此, 只需要按从高到低的方式清除 skyline, 直到剩下一个最高的 skyline 并且它的 endposition 大于当前 position
            while l >= hp[0][1]: 
                heapq.heappop(hp)
            
            # 对于高度增加到 h 的时间(neg_h < 0), 我们需要添加一个 skyline, 他将持续到 r 即 endposition
            if neg_h:
                heapq.heappush(hp, (neg_h, r))
            
            # 由于 res[-1][1] 记录了在当前事件触发之前一直保持的 skyline
            # 如果当前事件触发后 skyline 发生了改变
            #     1. 来了一条新的高度大于 h 的 skyline
            #     2. res[-1] 中记录的 skyline 到达了 endposition
            # 这两种事件都会导致刚才持续的 skyline 与现在最高的 skyline 不同; 同时, `key point` 产生了, 他将被记录在 res 中 
            if res[-1][1] != -hp[0][0]:
                res.append([l, -hp[0][0]])
        
        return res[1:]
```

#### template and summary of interval operations

In [ ]:
# interval intersection
s_in, t_in = max(s1, s2), min(t1, t2)
if t_in > s_in: return [s_in, t_in]

# remove interval [rs, rt] from interval [s, t]
# there are four cases: [1,4] remove [2,3], [1,4] remove [0,2], [1,4] remove [3,5], [1,4] removes [0,5] 
# method one
s_1, t_1 = s, max(s_in, s)
s_2, t_2 = min(t_in, t), t

if t_1 > s_1:
    res.append([s_1, t_1])
if t_2 > s_2:
    res.append([s_2, t_2])

# method two
if i[1] < toBeRemoved[0] or i[0] > toBeRemoved[1]:
    pass
else:
    if i[0] < toBeRemoved[0]:
        res.append([i[0], toBeRemoved[0]])
    # If there is some part of the interval after toBeRemoved, add that part
    if i[1] > toBeRemoved[1]:
        res.append([toBeRemoved[1], i[1]])
# 如果两个都相等 就整个[s, t]被删除了

# one interval is out of the other
if s_1 > t_2 or s_2 > t_1:
    pass

# merged interval [s, t], [s', t']
sm, tm = [min(s, s'), max(t, t')]

In [1]:
def insert(intervals, new_interval):
    res = []
    for cur in intervals:
        if new_interval is None or cur[1] < new_interval[0]: # 已经把new interval加进去了 只需要copy所有原来的interval到res
            res.append(cur)
        elif cur[0] > new_interval[1]: # interval 比当前的interval要靠前 add new interval before current interval
            res.append(new_interval)
            new_interval = None
            res.append(cur)
        else:
            new_interval[0] = min(new_interval[0], cur[0]) # merge intervals but not added to res
            new_interval[1] = max(new_interval[1], cur[1])
    
    if new_interval is not None:
        res.append(new_interval)
    
    return res

# Example usage:
intervals = [[1,3], [6,9]]
new_interval = [2,5]
print(insert(intervals, new_interval))

def remove_interval(intervals, to_be_removed):
    res = []
    for i in intervals:
        # If no overlap, add the interval as is
        if i[1] < to_be_removed[0] or i[0] > to_be_removed[1]:
            res.append(i)
        else:
            # If there is some part of the interval before to_be_removed, add that part
            if i[0] < to_be_removed[0]:
                res.append([i[0], to_be_removed[0]])
            # If there is some part of the interval after to_be_removed, add that part
            if i[1] > to_be_removed[1]:
                res.append([to_be_removed[1], i[1]])
            # 如果两个都相等 就整个[s, t]被删除了
            # 注意相交有四种形态 s, rs, t, rt 或者 rs, s, rt, t 或者 rs, s, t, rt. s, rs, rt, t
    return res

    def removeInterval(intervals: List[List[int]], toBeRemoved: List[int]) -> List[List[int]]:

        res = []
        rs, rt = toBeRemoved
        for (s,t) in intervals:
            # intersect
            s_in, t_in = max(s, rs), min(t, rt)
            if s_in >= t_in:
                res.append([s, t])
                continue
            # removed
            s1, t1 = s, max(s, s_in)
            s2, t2 = min(t_in, t), t

            if t1-s1>0:
                res.append([s1, t1])
            if t2-s2 >0:
                res.append([s2, t2])
        return res
    def removeInterval(intervals: List[List[int]], toBeRemoved: List[int]) -> List[List[int]]:
        ss = [x[0] for x in intervals]
        ee = [x[1] for x in intervals]
        x,y = toBeRemoved
        ans = []
        idx = bisect_left(ee, x)
        ans += intervals[:idx]
        l = intervals[idx][0]
        r = min(x, intervals[idx][1])
        if l < r:
            ans += [[l, r]]
        idx = bisect_left(ss, y)
        if idx == 0: return intervals
        l = min(y, intervals[idx-1][1])
        r = intervals[idx-1][1]
        if l < r:
            ans += [[l, r]]
        # 
        ans += intervals[idx:]
        return ans
# Example usage:
intervals = [[1, 2], [3, 4], [5, 6]]
to_be_removed = [4, 5]
print(remove_interval(intervals, to_be_removed))  # Expected output would be the intervals excluding the removed part


[[1, 5], [6, 9]]


# summary
Interval question template : 数飞机
一般两种解法：使用pq (heap)或者将start/end分开进行扫描线
考点: sort常用comparator的写法, 判断interval merge的边界条件
# 扫描线模版 dict-> add 1 minus 1 -> scan sorted keys
def scan_merged_intervals(intervals):
    dct = defaultdict(int)
    for s, t in intervals:
        dct[s] += 1
        dct[t] -= 1
    
    cnt = 0
    for time in sorted(dct.keys()):
        cnt += dct[time]
        if cnt >= k:
            return False
    return True
# heapq template: scan sorted start -> heapify end -> pop end before next start
def heap_merged_intervals(intervals):
    # 按照开始时间排序
    intervals = sorted(intervals, key=lambda x: x[0]) #intervals = sorted(intervals, key=x[0])
    # 按照结束时间建造heap
    hq = [float("inf")]
    heapq.heapify(hq)
    #hq = heapq.heapify([float("inf")]) #这样hq是None
    cnt = 0
    for s, t in intervals:
        while hq and hq[0] <= s: # 注意hq最小值的index是0
            heapq.heappop(hq)
        
        heapq.heappush(hq, t)
        cnt = max(cnt, len(hq)-1)
    return cnt


In [ ]:
# 

from typing import List

class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end

class Point:
    def __init__(self, time, type):
        self.time = time
        self.type = type

def count_of_airplanes(airplanes: List[Interval]) -> int:
    points = []
    for i in airplanes:
        points.append(Point(i.start, 1))
        points.append(Point(i.end, 0))
    
    points.sort(key=lambda p: (p.time, p.type))
    
    cnt = 0
    ans = 0
    for p in points:
        if p.type == 1:
            cnt += 1
        else:
            cnt -= 1
        ans = max(ans, cnt)
    
    return ans

# Example usage
airplanes = [Interval(1, 10), Interval(2, 3), Interval(5, 8), Interval(4, 7)]
print(count_of_airplanes(airplanes))  # Expected to print the maximum number of airplanes in the sky


In [3]:
# Employee Free Time
from collections import defaultdict
def solution(schedule):
    # TODO 尝试一下 interval intersection 的办法
    # method 1 sweep line
    d_schedule = defaultdict(int)
    for emp, ins in enumerate(schedule):
        if ins == []: return []
        for s, t in ins:
            d_schedule[s] += 1
            d_schedule[t] -= 1

    precnt, cnt = 0, 0
    res = []
    cs, ct = -1, 0
    for time, occupy in sorted(d_schedule.items()):
        cnt += occupy
        if precnt == 0 and cnt > 0:
            ct = time
            if ct > cs and cs != -1:
                res.append([cs, ct])
        elif cnt == 0:
            cs = time
        precnt = cnt
    return res

intervals = [[[1,2],[5,6]],[[1,3]],[[4,10]]]
intervals = [[[1,3],[6,7]],[[2,4]],[[2,5],[9,12]]]
intervals = [[[45,56],[89,96]],[[5,21],[57,74]]]
res = solution(intervals)
print(res)

[[21, 45], [56, 57], [74, 89]]


In [22]:
import heapq
from typing import *

def solution(buildings: List[List[int]]) -> List[List[int]]:

    H = [ [0, float("inf"), -1]] # [height, right, flag]

    heapq.heapify(H)
    res = [] # [-curh, e]
    curr = 0
    curh = 0
    preh = 0
    # TODO B = buildings.sort(key=lambda x: (x[0], -x[1]))
    B = buildings
    B.append([float("inf"), float("inf"), 0])
    B.sort(key = lambda x: (x[0], -x[1]))

    for i in range(len(B)):
        # x is a turning point if current height is different from previous height
        s, e, h = B[i]
        # push first
        # push building b into heap H
        # heapq.heappush(H, [-h, e, -1])
        # pop items from H with ends prior than s
        while len(H) > 1 and H[0][1] <= s:
            oh, oe, of = heapq.heappop(H)
            curr = max(curr, oe)
            curh = -H[0][0] if H[0][1] > curr else curh
            if curh != preh and H[0][1] > curr: # curr, curh is a turning point
                res.append([curr, curh])
                preh = curh

        heapq.heappush(H, [-h, e, -1])
        if res and s == res[-1][0]:
            res.pop(-1)
            if res and h != res[-1][1]:
                res.append([s, h])
        curh = -H[0][0]
        # check if there is a turning point
        if curh!= preh:
            res.append([s, curh])
            preh = curh

    return res

cases = [
    #[[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]],
    [[0,2,3],[2,5,3]]

]
for case in cases:
    res = solution(case)
    print(res)


[[0, 3], [2, 3], [5, 0]]


In [35]:
ints = [[1,3], [4,6], [8,12], [3, 5]]
import bisect
val = 4
i1 = bisect.bisect_left(ints, [val, 7])
i2 = bisect.bisect_right(ints, [val, 6])
i3 = bisect.bisect_right(ints, [val, 6])
print(i1, i2, i3)
print(ints[i1], ints[i2], ints[i3])

2 2 2
[8, 12] [8, 12] [8, 12]


In [ ]:
from sortedcontainers import SortedList
class Solution:
    def getSkyline(self, buildings: List[List[int]]) -> List[List[int]]:
        if len(buildings) == 0: 
            return []
        
        buildings.sort(key=lambda v: v[2])
        pos, height = [0], [0]
        for left, right, h in buildings: 
            i = bisect_left(pos, left)
            j = bisect_right(pos, right)
            height[i:j] = [h, height[j-1]]
            pos[i:j] = [left, right]
        print(height, pos)
        res = []
        prev = 0
        for v, h in zip(pos, height): 
            if h != prev:
                res.append([v,h]) 
                prev = h
                
        return res

In [ ]:
from heapq import heappush, heappop
class Solution(object):
    def getSkyline(self, buildings):
        # add start-building events
        # also add end-building events(acts as buildings with 0 height)
        # and sort the events in left -> right order
        events = [(L, -H, R) for L, R, H in buildings]
        events += list({(R, 0, 0) for _, R, _ in buildings})
        events.sort()

        # res: result, [x, height]
        # live: heap, [-height, ending position]
        res = [[0, 0]]
        live = [(0, float("inf"))]
        for pos, negH, R in events:
            # 1, pop buildings that are already ended
            # 2, if it's the start-building event, make the building alive
            # 3, if previous keypoint height != current highest height, edit the result
            while live[0][1] <= pos: heappop(live)
            if negH: heappush(live, (negH, R))
            if res[-1][1] != -live[0][0]:
                res += [ [pos, -live[0][0]] ]
        return res[1:]

In [ ]:
class Solution:
    def getSkyline(self, buildings):
        """
        :type buildings: List[List[int]]
        :rtype: List[List[int]]
        """
        # 对于一个 building, 他由 (l, r, h) 三元组组成, 我们可以将其分解为两种事件: 
        #     1. 在 left position, 高度从 0 增加到 h(并且这个高度将持续到 right position); 
        #     2. 在 right position, 高度从 h 降低到 0. 
        # 由此引出了 event 的结构: 在某一个 position p, 它引入了一个高度为 h 的 skyline, 将一直持续到另一 end postion
        
        # 对于在 right position 高度降为 0 的 event, 它的持续长度时无效的
        # 只保留一个 right position event, 就可以同时触发不同的两个 building 在同一 right position 从各自的 h 降为 0 的 event, 所以对 right position events 做集合操作会减少计算量
        
        # 由于需要从左到右触发 event, 所以按 postion 对 events 进行排序
        # 并且, 对于同一 positon, 我们需要先触发更高 h 的事件, 先触发更高 h 的事件后, 那么高的 h 相比于低的 h 会占据更高的 skyline, 低 h 的 `key point` 就一定不会产生; 相反, 可能会从低到高连续产生冗余的 `key point`
        # 所以, event 不仅需要按第一个元素 position 排序, 在 position 相同时, 第二个元素 h 也是必须有序的
        events = sorted([(l, -h, r) for l, r, h in buildings] + list({(r, 0, 0) for l, r, h in buildings}))
        
        # res 记录了 `key point` 的结果: [x, h]
        # 同时 res[-1] 处的 `key point` 代表了在下一个 event 触发之前, 一直保持的最高的 skyline
        
        # hp 记录了对于一条高为 h 的 skyline, 他将持续到什么 position 才结束: [h, endposition]
        # 在同时有多条 skyline 的时候, h 最高的那条 skyline 会掩盖 h 低的 skyline, 因此在 event 触发时, 需要得到当前最高的 skyline
        # 所以利用 heap 结构存储 hp, 它的第一个值永远为列表中的最小值: 因此在 event 中记录的是 -h, heap 结构就会返回最高的 skyline. 同时, h 必须在 endposition 之前, 因为它按第一个元素排序
        res, hp = [[0, 0]], [(0, float('inf'))]

        for l, neg_h, r in events:
            # 触发 event 时, 首先要做的就是清除已经到 endposition 的 skyline
            # hp: [h, endposition]
            # 如果当前 position 大于等于了 hp 中的 endposition, 那么该 skyline 会被清除掉
            # 由于在有 high skyline 的情况下, low skyline 不会有影响, 因此, 只需要按从高到低的方式清除 skyline, 直到剩下一个最高的 skyline 并且它的 endposition 大于当前 position
            while l >= hp[0][1]: 
                heapq.heappop(hp)
            
            # 对于高度增加到 h 的时间(neg_h < 0), 我们需要添加一个 skyline, 他将持续到 r 即 endposition
            if neg_h:
                heapq.heappush(hp, (neg_h, r))
            
            # 由于 res[-1][1] 记录了在当前事件触发之前一直保持的 skyline
            # 如果当前事件触发后 skyline 发生了改变
            #     1. 来了一条新的高度大于 h 的 skyline
            #     2. res[-1] 中记录的 skyline 到达了 endposition
            # 这两种事件都会导致刚才持续的 skyline 与现在最高的 skyline 不同; 同时, `key point` 产生了, 他将被记录在 res 中 
            if res[-1][1] != -hp[0][0]:
                res.append([l, -hp[0][0]])
        
        return res[1:]